In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import h5py
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D
from sklearn.model_selection import train_test_split
import sys
from sklearn import metrics
from keras.models import load_model
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as skAUC
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, TensorBoard
import tensorflow as tf
from keras import backend as K
import time
import glob
from tqdm import tqdm
import os
import csv

In [2]:
import pathlib
from keras import backend as K 
# tf.config.experimental.list_physical_devices('GPU')
path = pathlib.Path.cwd()
path

PosixPath('/local_disk/james/Taylor_data/Train/CNN_architecture')

In [3]:

with open('./best_all_CNN_model_0p3.csv', 'r') as f:
    reader = csv.reader(f)
    my_list = list(reader)
hpdirc = {my_list[0][i]:[my_list[1][i]] for i in range(len(my_list[0]))}


In [4]:
hpdirc

{'learning_rate': ['0.0004'],
 'convk_1': ['4'],
 'convp_1': ['200'],
 'pooling_1': ['3'],
 'convk_2': ['1'],
 'convp_2': ['248'],
 'convk_3': ['2'],
 'convp_3': ['192'],
 'units_1': ['320'],
 'Dropout_1': ['0.3'],
 'units_2': ['96'],
 'Dropout_2': ['0.1']}

In [5]:
def get_model(hp):
    inputs = tf.keras.Input(shape=(32,32))
  
    lr = float(hp['learning_rate'][0])
#     batch = hp.Int('batch_size', min_value=32, max_value=512, step=16)
    convk1 = int(hp['convk_1'][0])
    convp1 = int(hp['convp_1'][0])
    
    pool1 = int(hp['pooling_1'][0])
    
    convk2 = int(hp['convk_2'][0])
    convp2 = int(hp['convp_2'][0])
    
    
    convk3 = int(hp['convk_3'][0])
    convp3 = int(hp['convp_3'][0])
    
    dns1 = int(hp['units_1'][0])
    dp1 = float(hp['Dropout_1'][0])
    dns2 = int(hp['units_2'][0])
    dp2 = float(hp['Dropout_2'][0])
    
    
    x = inputs
    x = tf.expand_dims(x,axis=-1)
    x = get_filter(1)(x)
    x = tf.keras.layers.Conv2D(convp1, kernel_size=(convk1, convk1), activation='relu', input_shape=(32,32,1))(x)
#     x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = tf.keras.layers.MaxPooling2D(pool_size=(pool1, pool1))(x)
    
#     x = tf.keras.layers.Conv2D(56, kernel_size=(3,3), activation='relu', input_shape=(64,64,32))(x)
    x = tf.keras.layers.Conv2D(convp2, kernel_size=(convk2, convk2), activation='relu')(x)
    
#     x = tf.keras.layers.Conv2D(56, kernel_size=(3,3), activation='relu', input_shape=(15,15,64))(x)
    x = tf.keras.layers.Conv2D(convp3, kernel_size=(convk3, convk3), activation='relu')(x)
    
    x = tf.keras.layers.Flatten()(x)
    

    x = tf.keras.layers.Dense(dns1, activation='relu')(x)
    x = tf.keras.layers.Dropout(dp1)(x)
    x = tf.keras.layers.Dense(dns2, activation='relu')(x)
    x = tf.keras.layers.Dropout(dp2)(x)
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    modelCNN_basis = tf.keras.Model(inputs=inputs, outputs=x, name='CNN_basis')
    modelCNN_basis.compile(optimizer=keras.optimizers.Adam(lr) ,
                     loss="binary_crossentropy",
                     metrics=['accuracy'])
    print(hp.values)
    modelCNN_basis.summary()
#     print("Droup out :", dp1, " ", dp2)
#     print("lr :", lr)
                           
    return modelCNN_basis

In [6]:
class get_filter(tf.keras.layers.Layer):
    def __init__(self, units=1):
        super(get_filter, self).__init__()
        self.units = units
        
    def build(self, input_shape):  # Create the state of the layer (weights)
        wq_init = tf.random_normal_initializer()
        self.ft = tf.Variable(initial_value=wq_init(shape=( input_shape[1], input_shape[2], self.units), dtype='float32'), trainable=True, name='Basisfinder_ft')
        self.shape = input_shape
        
    def call(self, inputs):  # Defines the computation from inputs to outputs
        b = tf.math.abs(self.ft)
        return inputs*b

In [7]:
modelCNN_basis = get_model(hpdirc)

<built-in method values of dict object at 0x7fc2e03fb360>
Model: "CNN_basis"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32)]          0         
_________________________________________________________________
tf_op_layer_ExpandDims (Tens [(None, 32, 32, 1)]       0         
_________________________________________________________________
get_filter (get_filter)      (None, 32, 32, 1)         1024      
_________________________________________________________________
conv2d (Conv2D)              (None, 29, 29, 200)       3400      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 200)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 248)         49848     
_________________________________________________________________

In [8]:

i = 3
opt = ["Adadelta", "SGD", "Adagrad", "Adam"]
optn = opt[i]
opts = {"Adadelta":[tf.keras.optimizers.Adadelta()], "SGD":[tf.keras.optimizers.SGD()], 
        "Adagrad":[tf.keras.optimizers.Adagrad()], "Adam":[tf.keras.optimizers.Adam(learning_rate = float(hpdirc['learning_rate'][0]))]}
optimizer =opts[optn][0]

# modelCNN.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
modelCNN_basis.compile(optimizer= optimizer, loss="binary_crossentropy", metrics=['accuracy'])

modelCNN_basis.summary()

Model: "CNN_basis"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32)]          0         
_________________________________________________________________
tf_op_layer_ExpandDims (Tens [(None, 32, 32, 1)]       0         
_________________________________________________________________
get_filter (get_filter)      (None, 32, 32, 1)         1024      
_________________________________________________________________
conv2d (Conv2D)              (None, 29, 29, 200)       3400      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 9, 9, 200)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 248)         49848     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 192)         19

In [12]:
model_type = "CNN_basis"
save_dir = './Keras_Tunner/'
model_name = '%s_model.test.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

# lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)

# progress_bar = keras.callbacks.ProgbarLogger()

# csv_logger = keras.callbacks.CSVLogger(save_dir+'CNN'+rinv+'.csv')
# csv_logger = keras.callbacks.CSVLogger(save_dir+'CNN_'+rinv+'_'+optn+'_filter.csv')
csv_logger = keras.callbacks.CSVLogger(save_dir+'CNN_all_'+rinv+'_'+optn+'_filter.csv')


earlystop = tf.keras.callbacks.EarlyStopping(
                            monitor='val_loss',
                            min_delta=1e-4,
                            patience=3,
                            verbose=1,
                            mode='min', baseline=None, ## 'min' 
                            restore_best_weights=True)
# reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
#                               patience=2, min_lr=0.00001)
callbacks = [checkpoint, csv_logger,  earlystop ]


NameError: name 'rinv' is not defined

In [ ]:
modelCNN_basis.fit(Xim_train, yim_train , validation_data=(Xim_val, yim_val), callbacks = callbacks, shuffle=True , epochs=400, verbose=0)